In [ ]:
!pip install keras
!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from keras.utils import np_utils

wine = datasets.load_wine()

df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['target'] = wine.target
df[::20]

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
20,14.06,1.63,2.28,16.0,126.0,3.00,3.17,0.24,2.10,5.65,1.09,3.71,780.0,0
40,13.56,1.71,2.31,16.2,117.0,3.15,3.29,0.34,2.34,6.13,0.95,3.38,795.0,0
60,12.33,1.10,2.28,16.0,101.0,2.05,1.09,0.63,0.41,3.27,1.25,1.67,680.0,1
80,12.00,0.92,2.00,19.0,86.0,2.42,2.26,0.30,1.43,2.50,1.38,3.12,278.0,1
100,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0,1
120,11.45,2.40,2.42,20.0,96.0,2.90,2.79,0.32,1.83,3.25,0.80,3.39,625.0,1
140,12.93,2.81,2.70,21.0,96.0,1.54,0.50,0.53,0.75,4.60,0.77,2.31,600.0,2
160,12.36,3.83,2.38,21.0,88.0,2.30,0.92,0.50,1.04,7.65,0.56,1.58,520.0,2


In [3]:
# 必要なデータを抽出
X = df.drop(['target'], axis=1)
Y = df['target']

In [4]:
# dfをarrayに変換
X_array = np.array(X)
Y_array = np.array(Y)

In [5]:
# [0, 1, 2]を多クラスに分ける（0なら[1, 0, 0], 1なら[0, 1, 0], ...）
Y_array = np_utils.to_categorical(Y_array)

In [6]:
Y_array[::20]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split

# 学習データと検証データに分ける
train_x, test_x, train_y, test_y = train_test_split(X_array, Y_array, train_size=0.8, test_size=0.2, random_state=144)
len(train_x), len(test_x), len(train_y), len(test_y) # サイズ表示

(142, 36, 142, 36)

## Sequential Model

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(input_dim=13, units=3)) # 入力が13種の特徴、3つに分離
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.1))
model.fit(train_x, train_y, epochs=100, batch_size=10) # エポック100, ミニバッチ 10

Epoch 1/100
15/15 [==============================] - 14s 2ms/step - loss: 14412.0906
Epoch 2/100
15/15 [==============================] - 0s 3ms/step - loss: 20321.6794
Epoch 3/100
15/15 [==============================] - 0s 3ms/step - loss: 29209.0529
Epoch 4/100
15/15 [==============================] - 0s 3ms/step - loss: 17791.1550
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 13242.4053
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 16024.6640
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 22094.3163
Epoch 8/100
15/15 [==============================] - 0s 3ms/step - loss: 17506.9810
Epoch 9/100
15/15 [==============================] - 0s 3ms/step - loss: 13149.4514
Epoch 10/100
15/15 [==============================] - ETA: 0s - loss: 7678.39 - 0s 2ms/step - loss: 8751.7852
Epoch 11/100
15/15 [==============================] - 0s 2ms/step - loss: 12617.2024
Epoch 12/100
15/15 [============================

Epoch 98/100
15/15 [==============================] - 0s 3ms/step - loss: 5264.5920
Epoch 99/100
15/15 [==============================] - 0s 2ms/step - loss: 14031.4123
Epoch 100/100
15/15 [==============================] - 0s 2ms/step - loss: 4794.7815


In [9]:
_, predict_y = np.where(0 < (model.predict(test_x, batch_size=10) > 0.5).astype('int32'))
predict_y

array([2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 1, 1, 2,
       0, 1, 2, 1, 1, 2, 0, 2, 0, 1, 1, 2, 1, 2])

In [10]:
# 正解データの取得
_, Y_index = np.where(test_y > 0)
print(Y_index)

[1 1 0 0 0 1 2 2 1 0 0 0 2 2 1 2 1 1 2 2 1 1 0 1 2 1 1 1 0 1 0 1 1 2 1 1]


In [11]:
print('True:', np.count_nonzero(predict_y == Y_index))
print('False:', np.count_nonzero(predict_y != Y_index))

True: 24
False: 12


In [12]:
from sklearn.metrics import classification_report

print(classification_report(Y_index, predict_y))

              precision    recall  f1-score   support

           0       0.78      0.78      0.78         9
           1       0.73      0.61      0.67        18
           2       0.50      0.67      0.57         9

    accuracy                           0.67        36
   macro avg       0.67      0.69      0.67        36
weighted avg       0.69      0.67      0.67        36



## Random Forest Model

In [13]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=144)
clf.fit(train_x, train_y)

DecisionTreeClassifier(random_state=144)

In [14]:
# 予測結果
_, predict_y = np.where(0 < clf.predict(test_x))
predict_y

array([1, 1, 0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 1, 1,
       0, 1, 2, 1, 1, 1, 0, 1, 0, 2, 1, 2, 1, 1])

In [15]:
# 正解データの取得
_, Y_index = np.where(test_y > 0)
print(Y_index)

[1 1 0 0 0 1 2 2 1 0 0 0 2 2 1 2 1 1 2 2 1 1 0 1 2 1 1 1 0 1 0 1 1 2 1 1]


In [16]:
print('True:', np.count_nonzero(predict_y == Y_index))
print('False:', np.count_nonzero(predict_y != Y_index))

True: 32
False: 4


In [17]:
from sklearn.metrics import classification_report

print(classification_report(Y_index, predict_y))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      0.78      0.88        18
           2       0.75      1.00      0.86         9

    accuracy                           0.89        36
   macro avg       0.88      0.93      0.89        36
weighted avg       0.91      0.89      0.89        36

